In [1]:
%pylab
import random
from matplotlib  import cm
from matplotlib import colors
import random
import math
colormap = plt.cm.jet

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
def energy(x,y):
    aa = [-1.0,-1.0,-6.5,0.7]
    bb = [0.0,0.0,11.0,0.6]
    cc = [-10.0,-10.0,-6.5,0.7]
    AA = [-200.0,-100.0,-170.0,15.0]
    XX = [1.0,0.0,-0.5,-1.0]
    YY = [0.0,0.5,1.5,1.0]
    #energy=np.zeros((np.size(x),np.size(y)))
    energy = AA[0]*np.exp(aa[0]*np.square(x-XX[0])+bb[0]*np.multiply(x-XX[0],y-YY[0])+cc[0]*np.square(y-YY[0]))
    energy += AA[1]*np.exp(aa[1]*np.square(x-XX[1])+bb[1]*np.multiply(x-XX[1],y-YY[1])+cc[1]*np.square(y-YY[1]))
    energy += AA[2]*np.exp(aa[2]*np.square(x-XX[2])+bb[2]*np.multiply(x-XX[2],y-YY[2])+cc[2]*np.square(y-YY[2]))
    energy += AA[3]*np.exp(aa[3]*np.square(x-XX[3])+bb[3]*np.multiply(x-XX[3],y-YY[3])+cc[3]*np.square(y-YY[3]))
        #energy=np.add(energy,et)
    return energy

In [13]:
size=50
x=np.linspace(-2, 1.5,size)
y=np.linspace(-1, 2.5,size)

xv, yv = np.meshgrid(x, y, sparse=False, indexing='xy')
e=energy(xv,yv)
plt.figure(1)
normalize = colors.Normalize(vmin=np.min(e), vmax=200)
plt.scatter(xv,yv,c=e,cmap = colormap,norm=normalize,s=4)
plt.ylim(-1,2.5)
plt.xlim(-2,1.5)

(-2, 1.5)

In [14]:
print(np.min(e))
result = numpy.where(e == numpy.amin(e))
r0 = list(zip(result[0],result[1]))
x0 = xv[r0[0][0]][[r0[0][1]]]
y0 = yv[r0[0][0]][r0[0][1]]
print(energy(x0,y0))

-146.6286319083252
[-146.62863191]


In [15]:
left=[1,0,0,0]
right=[0,1,0,0]
up =[0,0,1,0]
down = [0,0,0,1]
possible_actions = [left,right,up,down]

state_size = 2
action_size = 4
learning_rate = 0.0002



In [17]:
class DQModel:
    def __init__(self, state_size,action_size,learning_rate,name="DQModel"):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        
        with tf.variable_scope(name):
            self.inputs_ = tf.placeholder(tf.float32,[None, *state_size],name="inputs")
            self.actions_ = tf.placeholder(tf.float32,[None, 4],name="actions_")
            
            self.target_Q = tf.placeholder(tf.float32,[None],name='target')
            
            self.d1 = tf.layers.Dense(inputs=self.inputs_,
                                      units = 20,
                                      activation='sigmoid',
                                      kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                      name="d1")
            
            self.d2 = tf.layers.Dense(inputs=self.d1,
                                      units = 20,
                                      activation='sigmoid',
                                      kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                      name="d2")
            
            self.output = tf.layers.Dense(inputs = self.d2, 
                                          units=4,
                                          activation='linear',
                                          kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                          name="output")
            
            self.Q = tf.reduce_sum(tf.multiply(self.output, self.actions_), axis=1)
            
            self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q))
            
            self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)
            
            

In [111]:
tf.reset_default_graph()
DQModel = DQModel(state_size, action_size, learning_rate)


[[6.94725716e-310 6.94725716e-310]
 [4.65242396e-310 4.65242396e-310]
 [6.94720202e-310 6.94725187e-310]
 ...
 [0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000]]
[[-0.54545455  1.45454545]
 [-0.54545455  1.45454545]
 [-0.54204689  1.43688273]
 ...
 [-0.57543111  1.43691655]
 [-0.57543111  1.43691655]
 [-0.57543111  1.43691655]]


In [ ]:
nsteps = 1000
dx = 0.2

e_old = energy(x0,y0)
beta = 1.0

xO = x0
yO = y0
acc_rate = 0
traj = np.empty((nsteps,2))
print(traj)
for i in range (nsteps):
    xN = xO + (random.uniform(0,1)-0.5)*dx/math.sqrt(2)
    yN = yO + (random.uniform(0,1)-0.5)*dx/math.sqrt(2)
    
    e = energy(xN,yN)
    dE = e - e_old 
    acc = False
    
    if(dE <= 0):
        acc = True
    else:
        P = exp(-beta*dE)
        if P >= random.uniform(0,1):
            acc = True
        else:
            acc = False
    
    if acc:
        xO = xN
        yO = yN
        e_old = e
        acc_rate += 1
    
    traj[i][0] = xO
    traj[i][1] = yO
        
print(traj) 
e=energy(xv,yv)
plt.scatter(xv,yv,c=e,cmap = colormap,norm=normalize)
plt.scatter(traj[:,0],traj[:,1],s=2,c='white')